# Detecting Coordination in Civic Discourse

- See [01_coordination_detection.md](../docs/01_coordination_detection.md)
- See [02_limitations.md](../docs/02_limitations.md)
- See [03_coordination_actors.md](../docs/03_coordination_actors.md)

### How Can We Assess Behavior without Reading Content

- See [04_content_hashes.md](../docs/04_content_hashes.md)

### The Detection Approach (Assess Behavior, Not Content or Posters)

- See [05_behavior_and_detection.md](../docs/05_behavior_and_detection.md)

## Your Mission

You have access to transparency data from **two social media discourse datasets** during local political discussions.

- **World A**: Contains transparency data from one world
- **World B**: Contains transparency data from another world

**Your goal**: Determine which world shows organic civic discourse and which shows signs of coordinated manipulation.

Use only behavioral patterns (how much reposting, how new are the accounts, how "bursty" is the activity) - no direct inspection of the content or the people posting is needed.

### City Calendar (Interesting Events) and Simulation Scope

- See [06_calendar_scope.md](../docs/06_calendar_scope.md)

## SETUP: Imports & Connect to the Databases

As always, we'll start by importing all the packages needed for our notebook. 
We'll also open a connection to the two databases, one representing World A and one representing World B. 

In [37]:
# imports at the beginning
import warnings

import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

warnings.filterwarnings('ignore')

# Connect to both datasets
world_a = duckdb.connect('../data/worlds/civic_world_a.duckdb')
world_b = duckdb.connect('../data/worlds/civic_world_b.duckdb')

print("Connected to civic transparency datasets")
print("World A data points:", world_a.execute("SELECT COUNT(*) FROM transparency_points").fetchone()[0])
print("World B data points:", world_b.execute("SELECT COUNT(*) FROM transparency_points").fetchone()[0])

Connected to civic transparency datasets
World A data points: 13500
World B data points: 13500


## Explore the Data: What Topics Are Available?

These are represented by a unique hash value - we don't need to see what they're saying, instead we find posts and use "fuzzy hashing" to see what posts are essentially very similar. Privacy is important, so we'll call each topic by its unique hash value. 

Analysts have done some **prior work** and estimated what each hash value might represent based on the City Calendar and the posting timing and behavior:

- See their estimated mappings at [07_prior_work.md](./07_prior_work.md).
- For data definitions, see [08_data_definitions.md](./08_data_definitions.md)

In [38]:
# Show unique topics in World A
print("\nWorld A Topics:")
topics_a = world_a.execute("SELECT DISTINCT topic FROM transparency_series").df()
print(f"  {len(topics_a)} topics")
print(topics_a.head(15))


World A Topics:
  15 topics
       topic
0   i9f8e1c7
1   j2a9f5d3
2   k6e4b7f1
3   c9f6a3d8
4   g7c2f9b4
5   m4f7e3b8
6   e8b3c6f9
7   d4e7f2a1
8   n1d5a8f2
9   b8e4c1f5
10  a7f3d9e2
11  f1a5d8e2
12  h3d6a4e8
13  l8c1d9a6
14  o7b9e4c1



### Pre-built Analysis Views

Each world database includes pre-built SQL views that aggregate the data for easier analysis:

- **vw_coordination_summary** - Topic-level coordination metrics
- **vw_manipulation_indicators** - High-scoring suspicious events
- **vw_account_behavior** - Account age vs automation patterns
- **vw_content_patterns** - Content recycling and reshare patterns
- **vw_hourly_patterns** - Temporal activity patterns
- **vw_world_diagnosis** - Overall world statistics


In [39]:
# # List available views in World A
# print("\nAvailable Views:")
# views = world_a.execute("""
#     SELECT table_name 
#     FROM information_schema.tables 
#     WHERE table_schema = 'main' AND table_name LIKE 'vw_%'
#     ORDER BY table_name
# """).df()
# display(views)


## View 1: Coordination Patterns (Across All Topics)

Let's start by comparing the overall coordination patterns across all the selected topics.

In [40]:
# Get coordination summaries for both worlds
coord_a = world_a.execute("SELECT * FROM vw_coordination_summary ORDER BY suspicion_score DESC").df()
coord_b = world_b.execute("SELECT * FROM vw_coordination_summary ORDER BY suspicion_score DESC").df()

print("World A - Coordination Summary:")
print(coord_a[['topic', 'avg_burst_score', 'avg_synchrony_index', 'high_burst_count', 'suspicion_score']].head(5))

print("\nWorld B - Coordination Summary:")
print(coord_b[['topic', 'avg_burst_score', 'avg_synchrony_index', 'high_burst_count', 'suspicion_score']].head(5))

World A - Coordination Summary:
      topic  avg_burst_score  avg_synchrony_index  high_burst_count  \
0  h3d6a4e8            0.155                0.229                 0   
1  n1d5a8f2            0.153                0.230                 0   
2  d4e7f2a1            0.153                0.224                 0   
3  b8e4c1f5            0.150                0.228                 0   
4  a7f3d9e2            0.151                0.227                 0   

   suspicion_score  
0            0.187  
1            0.186  
2            0.185  
3            0.185  
4            0.184  

World B - Coordination Summary:
      topic  avg_burst_score  avg_synchrony_index  high_burst_count  \
0  a7f3d9e2            0.915                0.879               900   
1  l8c1d9a6            0.915                0.874               900   
2  b8e4c1f5            0.561                0.579               147   
3  c9f6a3d8            0.560                0.577               136   
4  i9f8e1c7            0.37

In [41]:
# Visual comparison of coordination signals
# Create box plots for each metric to show
# the distribution (min, max, median, quartiles) across all topics
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=(
        'Burst Scores (Sudden spikes)',
        'Synchrony Index (time-aligned posting)',
        'High Burst Events (many suspicious bursts)',
        'Overall Suspicion (calculated)'),
    vertical_spacing=0.15
)

# Use same y-axis scale for fair comparison
y_max_burst = max(coord_a['avg_burst_score'].max(), coord_b['avg_burst_score'].max()) * 1.1
y_max_sync = max(coord_a['avg_synchrony_index'].max(), coord_b['avg_synchrony_index'].max()) * 1.1
y_max_events = max(coord_a['high_burst_count'].max(), coord_b['high_burst_count'].max()) * 1.1
y_max_suspicion = max(coord_a['suspicion_score'].max(), coord_b['suspicion_score'].max()) * 1.1

# Burst scores
fig.add_trace(go.Box(y=coord_a['avg_burst_score'], name='World A', marker_color='blue'), row=1, col=1)
fig.add_trace(go.Box(y=coord_b['avg_burst_score'], name='World B', marker_color='lightcoral'), row=1, col=1)
fig.update_yaxes(range=[0, y_max_burst], row=1, col=1)

# Synchrony scores  
fig.add_trace(go.Box(y=coord_a['avg_synchrony_index'], name='World A', marker_color='blue', showlegend=False), row=1, col=2)
fig.add_trace(go.Box(y=coord_b['avg_synchrony_index'], name='World B', marker_color='lightcoral', showlegend=False), row=1, col=2)
fig.update_yaxes(range=[0, y_max_sync], row=1, col=2)

# High burst events
fig.add_trace(go.Box(y=coord_a['high_burst_count'], name='World A', marker_color='blue', showlegend=False), row=2, col=1)
fig.add_trace(go.Box(y=coord_b['high_burst_count'], name='World B', marker_color='lightcoral', showlegend=False), row=2, col=1)
fig.update_yaxes(range=[0, y_max_events], row=2, col=1)

# Suspicion scores
fig.add_trace(go.Box(y=coord_a['suspicion_score'], name='World A', marker_color='blue', showlegend=False), row=2, col=2)
fig.add_trace(go.Box(y=coord_b['suspicion_score'], name='World B', marker_color='lightcoral', showlegend=False), row=2, col=2)
fig.update_yaxes(range=[0, y_max_suspicion], row=2, col=2)

fig.update_layout(height=600, title="Coordination Comparison (Across All Topics)")
fig.show()

# Summary statistics
print("\nSummary Statistics:")
print(f"World A avg burst score: {coord_a['avg_burst_score'].mean():.3f}")
print(f"World B avg burst score: {coord_b['avg_burst_score'].mean():.3f}")
print("")
print(f"World A avg high burst events: {coord_a['high_burst_count'].mean():.1f}")
print(f"World B avg high burst events: {coord_b['high_burst_count'].mean():.1f}")
print("")
print(f"World A avg suspicion score: {coord_a['suspicion_score'].mean():.3f}")
print(f"World B avg suspicion score: {coord_b['suspicion_score'].mean():.3f}")


Summary Statistics:
World A avg burst score: 0.151
World B avg burst score: 0.344

World A avg high burst events: 0.0
World B avg high burst events: 138.9

World A avg suspicion score: 0.184
World B avg suspicion score: 0.364


## View 2. How Does Coordination Change Over Time (for a single topic)

Let's examine how coordination patterns change over time for a specific civic topic.

### 2.1 First, what topics are available? 

In [42]:
# Show condensed list of topics to explore from World A
print("\nWorld A Topics to Explore:")
topics_a = world_a.execute("SELECT DISTINCT topic FROM transparency_series").df()
# Display all topics in a single line separated by commas
print(", ".join(topics_a['topic']))



World A Topics to Explore:
b8e4c1f5, i9f8e1c7, j2a9f5d3, k6e4b7f1, d4e7f2a1, e8b3c6f9, a7f3d9e2, f1a5d8e2, h3d6a4e8, l8c1d9a6, o7b9e4c1, c9f6a3d8, n1d5a8f2, g7c2f9b4, m4f7e3b8


### 2.2 Choose a Topic and View the Timeline

TODO: **Replace topics in the code below** to explore each topic (as a hidden-content hash).

In [43]:

# Get hourly patterns for a sample topic with fixed scaling for fair comparison
sample_topic = "c9f6a3d8" # TODO: explore other topics from the list above

hourly_a = world_a.execute(f"SELECT * FROM vw_hourly_patterns WHERE topic = '{sample_topic}' ORDER BY hour").df()
hourly_b = world_b.execute(f"SELECT * FROM vw_hourly_patterns WHERE topic = '{sample_topic}' ORDER BY hour").df()

# Calculate consistent y-axis ranges for fair comparison
burst_max = max(hourly_a['avg_burst_score'].max(),hourly_b['avg_burst_score'].max()) * 1.1
volume_max = max(hourly_a['avg_volume'].max(), hourly_b['avg_volume'].max()) * 1.1

# Create two-row subplot with shared y-axes
# Left y-axis: Burst Score, Right y-axis: Volume
# Use secondary_y for right y-axis
# Use scatter for burst score (blue/coral points) and line for volume (grey trend)

fig = make_subplots(rows=2, cols=1, 
                   subplot_titles=(f'World A: {sample_topic}', 
                                 f'World B: {sample_topic}'),
                   specs=[[{"secondary_y": True}], [{"secondary_y": True}]])

# World A timeline
fig.add_trace(go.Scatter(x=hourly_a['hour'], y=hourly_a['avg_burst_score'], 
                        mode='lines+markers', name='Burst Score', line_color='blue'), 
              row=1, col=1, secondary_y=False)
fig.add_trace(go.Scatter(x=hourly_a['hour'], y=hourly_a['avg_volume'], 
                        mode='lines', name='Volume', line_color='grey', opacity=0.7), 
              row=1, col=1, secondary_y=True)

# World B timeline
fig.add_trace(go.Scatter(x=hourly_b['hour'], y=hourly_b['avg_burst_score'], 
                        mode='lines+markers', name='Burst Score', line_color='red'), 
              row=2, col=1, secondary_y=False)
fig.add_trace(go.Scatter(x=hourly_b['hour'], y=hourly_b['avg_volume'], 
                        mode='lines', name='Volume', line_color='grey', opacity=0.7), 
              row=2, col=1, secondary_y=True)

# Set consistent y-axis ranges
fig.update_yaxes(title_text="Burst Score", range=[0, burst_max], secondary_y=False)
fig.update_yaxes(title_text="Volume", range=[0, volume_max], secondary_y=True)

fig.update_layout(height=600, title="Temporal Patterns: Look for High Burst Scores and Volume Spikes")
fig.show()

print(f"\nTemporal Analysis for {sample_topic}:")
print(f"World A max burst score: {hourly_a['avg_burst_score'].max():.3f}")
print(f"World B max burst score: {hourly_b['avg_burst_score'].max():.3f}")
print(f"World A max volume: {hourly_a['avg_volume'].max():.0f}")
print(f"World B max volume: {hourly_b['avg_volume'].max():.0f}")


Temporal Analysis for c9f6a3d8:
World A max burst score: 0.164
World B max burst score: 0.586
World A max volume: 129
World B max volume: 236


## View 3. Explore Account Age and Automation

Examine the relationship between account age and automation usage - a key indicator of manipulation.

In [44]:
# Account behavior patterns
acct_a = world_a.execute("SELECT * FROM vw_account_behavior ORDER BY new_account_automation_correlation DESC").df()
acct_b = world_b.execute("SELECT * FROM vw_account_behavior ORDER BY new_account_automation_correlation DESC").df()

print("World A - Account Behavior (Top 5 topics):")
print(acct_a[['topic', 'avg_new_accounts', 'avg_api_posting', 'new_account_automation_correlation']].head())

print("\nWorld B - Account Behavior (Top 5 topics):")
print(acct_b[['topic', 'avg_new_accounts', 'avg_api_posting', 'new_account_automation_correlation']].head())

# Calculate consistent axis ranges
x_max = max(acct_a['avg_new_accounts'].max(), acct_b['avg_new_accounts'].max()) * 1.1
y_max = max(acct_a['avg_api_posting'].max(), acct_b['avg_api_posting'].max()) * 1.1

# Display side-by-side scatter plots with same scale
# Make the size of the bubbles proportional to the correlation value
# Larger bubbles = higher correlation
# The correlations range from 0.01 to 0.2
# Minimum bubble size = 10 pixels, maximum size = 50
min_size = 10
scale_factor = 50.0/0.2

# For World A
size_a = min_size + (acct_a['new_account_automation_correlation'] ** 0.7) * scale_factor

# For World B  
size_b = min_size + (acct_b['new_account_automation_correlation'] ** 0.7) * scale_factor

fig = make_subplots(rows=1, cols=2, 
                   subplot_titles=(
                       'A: New Accounts using API', 
                       'B: New Accounts using API'))

fig.add_trace(go.Scatter(x=acct_a['avg_new_accounts'], y=acct_a['avg_api_posting'], 
                        mode='markers', 
                        marker=dict(size=size_a, color='lightblue'),
                        text=acct_a['topic'], name='World A'), 
              row=1, col=1)

fig.add_trace(go.Scatter(x=acct_b['avg_new_accounts'], y=acct_b['avg_api_posting'], 
                        mode='markers',
                        marker=dict(size=size_b, color='lightcoral'),
                        text=acct_b['topic'], name='World B'), 
              row=1, col=2)

# Set consistent axis ranges
fig.update_xaxes(title_text="Avg New Accounts (0-7 days)", range=[0, x_max])
fig.update_yaxes(title_text="Avg API Posting", range=[0, y_max])

fig.update_layout(height=500, title="Account Age vs API Automation: Larger bubbles = higher correlation")
fig.show()

print("\nAutomation Correlation Analysis:")
print(f"World A avg correlation: {acct_a['new_account_automation_correlation'].mean():.3f}")
print(f"World B avg correlation: {acct_b['new_account_automation_correlation'].mean():.3f}")
print("\nHigher correlation (bubble sizes) indicate fraction")
print("of new accounts that are using automated posting tools.")

World A - Account Behavior (Top 5 topics):
      topic  avg_new_accounts  avg_api_posting  \
0  d4e7f2a1             0.098            0.098   
1  e8b3c6f9             0.097            0.099   
2  a7f3d9e2             0.099            0.100   
3  f1a5d8e2             0.099            0.099   
4  h3d6a4e8             0.098            0.099   

   new_account_automation_correlation  
0                                0.01  
1                                0.01  
2                                0.01  
3                                0.01  
4                                0.01  

World B - Account Behavior (Top 5 topics):
      topic  avg_new_accounts  avg_api_posting  \
0  a7f3d9e2             0.457            0.438   
1  l8c1d9a6             0.452            0.441   
2  c9f6a3d8             0.267            0.261   
3  b8e4c1f5             0.268            0.259   
4  i9f8e1c7             0.164            0.195   

   new_account_automation_correlation  
0                              


Automation Correlation Analysis:
World A avg correlation: 0.010
World B avg correlation: 0.047

Higher correlation (bubble sizes) indicate fraction
of new accounts that are using automated posting tools.


## View 4. How Coordinated is the Content? 

Let's use side-by-side box plots to review coordination between the two worlds. 

In [45]:
# Content patterns
content_a = world_a.execute("SELECT * FROM vw_content_patterns ORDER BY content_coordination_score DESC").df()
content_b = world_b.execute("SELECT * FROM vw_content_patterns ORDER BY content_coordination_score DESC").df()

print("World A - Content Patterns:")
print(content_a[['topic', 'avg_reshare_ratio', 'avg_recycled_content', 'high_recycling_points', 'content_coordination_score']].head())

print("\nWorld B - Content Patterns:")
print(content_b[['topic', 'avg_reshare_ratio', 'avg_recycled_content', 'high_recycling_points', 'content_coordination_score']].head())

# Compare content coordination scores
fig = go.Figure()
fig.add_trace(go.Box(y=content_a['content_coordination_score'], name='World A', marker_color='lightblue'))
fig.add_trace(go.Box(y=content_b['content_coordination_score'], name='World B', marker_color='lightcoral'))
fig.update_layout(title="Content Coordination Score Distribution", yaxis_title="Coordination Score")
fig.show()

print(f"\nContent Analysis:")
print(f"World A avg coordination score: {content_a['content_coordination_score'].mean():.3f}")
print(f"World B avg coordination score: {content_b['content_coordination_score'].mean():.3f}")

World A - Content Patterns:


      topic  avg_reshare_ratio  avg_recycled_content  high_recycling_points  \
0  g7c2f9b4              0.353                 0.101                      0   
1  h3d6a4e8              0.353                 0.100                      0   
2  o7b9e4c1              0.353                 0.101                      0   
3  c9f6a3d8              0.352                 0.101                      0   
4  n1d5a8f2              0.354                 0.098                      0   

   content_coordination_score  
0                       0.227  
1                       0.227  
2                       0.227  
3                       0.227  
4                       0.226  

World B - Content Patterns:
      topic  avg_reshare_ratio  avg_recycled_content  high_recycling_points  \
0  a7f3d9e2              0.873                 0.651                    900   
1  l8c1d9a6              0.874                 0.648                    900   
2  b8e4c1f5              0.651                 0.352              


Content Analysis:
World A avg coordination score: 0.226
World B avg coordination score: 0.354


## View 5. A Closer Look at Events with "High Manipulation" Scores

Let's examine the highest-scoring individual events that might indicate coordination.

In [46]:
# Top manipulation events from each world
manip_a = world_a.execute("SELECT * FROM vw_manipulation_indicators ORDER BY manipulation_score DESC LIMIT 8").df()
manip_b = world_b.execute("SELECT * FROM vw_manipulation_indicators ORDER BY manipulation_score DESC LIMIT 8").df()

print("World A - Top 8 Suspicious Events:")
print(manip_a[['topic', 'ts', 'volume', 'burst_score', 'synchrony_index', 'manipulation_score']].to_string(index=False))

print("\nWorld B - Top 8 Suspicious Events:")
print(manip_b[['topic', 'ts', 'volume', 'burst_score', 'synchrony_index', 'manipulation_score']].to_string(index=False))

print("\nManipulation Score Analysis:")
print(f"World A highest score: {manip_a['manipulation_score'].max():.3f}")
print(f"World B highest score: {manip_b['manipulation_score'].max():.3f}")
print(f"World A avg top-8 score: {manip_a['manipulation_score'].mean():.3f}")
print(f"World B avg top-8 score: {manip_b['manipulation_score'].mean():.3f}")

World A - Top 8 Suspicious Events:
   topic                  ts  volume  burst_score  synchrony_index  manipulation_score
l8c1d9a6 2026-02-03 20:39:00     156     0.243485         0.341885               0.256
k6e4b7f1 2026-02-03 20:01:00      61     0.245058         0.349634               0.256
l8c1d9a6 2026-02-01 11:18:00      87     0.246135         0.344162               0.254
j2a9f5d3 2026-02-03 20:00:00     146     0.249824         0.334791               0.252
b8e4c1f5 2026-02-01 20:55:00     173     0.222675         0.347477               0.251
m4f7e3b8 2026-02-01 09:11:00     155     0.249197         0.345561               0.251
c9f6a3d8 2026-02-01 09:09:00     186     0.245361         0.343014               0.250
c9f6a3d8 2026-02-03 08:23:00      50     0.237557         0.340458               0.250

World B - Top 8 Suspicious Events:
   topic                  ts  volume  burst_score  synchrony_index  manipulation_score
a7f3d9e2 2026-02-02 14:56:00     684     0.979941         0

## View 6. Comparing the Two Worlds (Overall Assessment)

In [47]:
# Overall diagnostic
diag_a = world_a.execute("SELECT * FROM vw_world_diagnosis").df()
diag_b = world_b.execute("SELECT * FROM vw_world_diagnosis").df()

print("WORLD COMPARISON SUMMARY")
print("=" * 50)

print("WORLD A:")
print(f"  Total topics: {diag_a['total_topics'].iloc[0]}")
print(f"  Data points: {diag_a['total_data_points'].iloc[0]:,}")
print(f"  Avg burst score: {diag_a['overall_avg_burst_score'].iloc[0]:.3f}")
print(f"  Avg synchrony: {diag_a['overall_avg_synchrony_index'].iloc[0]:.3f}")
print(f"  Coordinated events: {diag_a['coordinated_events'].iloc[0]}")
print(f"  Coordination percentage: {diag_a['coordination_percentage'].iloc[0]:.2f}%")

print("\nWORLD B:")
print(f"  Total topics: {diag_b['total_topics'].iloc[0]}")
print(f"  Data points: {diag_b['total_data_points'].iloc[0]:,}")
print(f"  Avg burst score: {diag_b['overall_avg_burst_score'].iloc[0]:.3f}")
print(f"  Avg synchrony: {diag_b['overall_avg_synchrony_index'].iloc[0]:.3f}")
print(f"  Coordinated events: {diag_b['coordinated_events'].iloc[0]}")
print(f"  Coordination percentage: {diag_b['coordination_percentage'].iloc[0]:.2f}%")

print("=" * 50)

# Calculate differences
coord_diff = diag_b['coordination_percentage'].iloc[0] - diag_a['coordination_percentage'].iloc[0]
burst_diff = diag_b['overall_avg_burst_score'].iloc[0] - diag_a['overall_avg_burst_score'].iloc[0]

print("\nKEY DIFFERENCES:")
print(f"Coordination % difference: {coord_diff:.2f} percentage points")
print(f"Burst score difference: {burst_diff:.3f}")
print(f"Coordinated events difference: {diag_b['coordinated_events'].iloc[0] - diag_a['coordinated_events'].iloc[0]}")

WORLD COMPARISON SUMMARY
WORLD A:
  Total topics: 15
  Data points: 13,500
  Avg burst score: 0.151
  Avg synchrony: 0.226
  Coordinated events: 0
  Coordination percentage: 0.00%

WORLD B:
  Total topics: 15
  Data points: 13,500
  Avg burst score: 0.344
  Avg synchrony: 0.398
  Coordinated events: 1861
  Coordination percentage: 13.79%

KEY DIFFERENCES:
Coordination % difference: 13.79 percentage points
Burst score difference: 0.193
Coordinated events difference: 1861


## Your Analysis and Conclusion

Look at how the different metrics compare between the two worlds and between different topics.
Do you see any evidence of possible manipulation (or lack thereof)?
Based on the patterns you've observed, answer these questions:

1. **Which world shows organic civic discourse?** Look for: Lower burst scores, fewer coordinated events, natural temporal patterns

2. **Which world shows signs of coordinated manipulation?** Look for: High burst scores, synchronized posting, suspicious account patterns

3. **What specific evidence supports your conclusions?** Point to concrete numbers and patterns from your analysis

4. **Which topics and timeframes were most suspicious?**

5. **If you were a platform moderator, would you investigate further? What would you look at?**

6. **Confidence level:** How certain are you? What could change your conclusion?


### Write your conclusions here:

[Space for analysis and conclusions]


## Want More Challenge?

- Compare specific topics (as content hashes) between worlds (which show the biggest difference?)
- Find the "most organic" and "most suspicious" topics in World B
- Design a real-time alert system: at what thresholds would you flag activity?

## References

- [references.md](../docs/references.md)
- [references.bib](../docs/references.bib)

In [48]:
# Clean up
world_a.close()
world_b.close()
print("Analysis complete. Database connections closed.")

Analysis complete. Database connections closed.
